## Packages

In [367]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import re

# Data prep

## Import data

In [368]:
## Import crime data
## Data extracted from: https://data.politie.nl/portal.html?_la=nl&_catalog=Politie&tableId=47013NED&_theme=114
## On: 21-12-2025
crime_data = 'allemisdrijven_pvgm_21122025.csv'
crime_df = pd.read_csv(crime_data, sep=';')
## Import crime metada
metadata = 'allemisdrijven_metadata_21122025.csv'
metadata_df = pd.read_csv(metadata, sep=';', header=1)

## Import population data from CBS
## Data extracted from: https://opendata.cbs.nl/#/CBS/nl/dataset/37230ned/table?ts=1766338546563
## On: 21-12-2025
popdata = 'bevolkingeindeperiod_jaren_22122025.csv'
popdata_df = pd.read_csv(popdata, sep=';', header=3)

In [369]:
metadata_df.head(20)

,ID,Title,ShortTitle,Identifier,Summary,Modified,ReasonDelivery,ExplanatoryText,Language,Catalog,...,Period,ShortDescription,Description,DefaultPresentation,DefaultSelection,GraphTypes,OutputStatus,Source,MetaDataModified,SearchPriority
0,0,Geregistreerde misdrijven en aangiften; soort ...,Misdrijven en aangiften per gemeente,47013NED,"Geregistreerde misdrijven, aangiften en intern...",2025-12-15T02:00:00,ActualiseringBijzonder,NaN,nl,Politie,...,2012-2025,\nDeze tabel bevat cijfers over het aantal ger...,INHOUDSOPGAVE\n\n1. Toelichting\n2. Definities...,ts=1764853734957&graphtype=Table&r=RegioS&k=To...,$filter=((SoortMisdrijf eq '0.0.0 ')) and ((Pe...,"Table,Bar,Map",Regulier,&copy; Politie,2025-12-15T02:00:00,2.0
1,DataProperties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID,Position,ParentID,Type,Key,Title,Description,ReleasePolicy,Datatype,Unit,...,Default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,NaN,Dimension,SoortMisdrijf,Soort misdrijf,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,NaN,GeoDimension,RegioS,Regio's,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,2,NaN,TimeDimension,Perioden,Perioden,NaN,TRUE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,3,NaN,Topic,GeregistreerdeMisdrijven_1,Geregistreerde misdrijven,"Totaal van de geregistreerde misdrijven, bron ...",NaN,Long,aantal,...,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,4,NaN,Topic,Aangiften_2,Aangiften,Aangiften betreffen geregistreerde misdrijven ...,NaN,Long,aantal,...,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5,5,NaN,Topic,Internetaangiften_3,Internetaangiften,Aangiften betreffen geregistreerde misdrijven ...,NaN,Long,aantal,...,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,SoortMisdrijf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [370]:
## Only need the first two columns with codes and their values
metadata_sub = metadata_df.iloc[9:,:2]
metadata_sub

,ID,Title
9,SoortMisdrijf,NaN
10,Key,Title
11,0.0.0,Totaal misdrijven
12,1.1.1,1.1.1 Diefstal/inbraak woning
13,1.1.2,1.1.2 Diefstal/inbraak box/garage/schuur
...,...,...
624,2025MM07,2025 juli
625,2025MM08,2025 augustus
626,2025MM09,2025 september
627,2025MM10,2025 oktober


In [371]:
popdata_df

,Regio's,Onderwerp,Unnamed: 2,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Nederland,Bevolking aan het einde van de periode,aantal,16192572.0,16258032.0,16305526.0,16334210.0,16357992.0,16405399.0,16485787.0,...,16979120.0,17081507.0,17181084.0,17282163.0,17407585.0,17475415.0,17590672.0,17811291.0,17942942.0,18044027.0
1,Groningen (PV),Bevolking aan het einde van de periode,aantal,572997.0,574384.0,575072.0,574042.0,573614.0,573459.0,574092.0,...,583721.0,583581.0,582944.0,583990.0,585866.0,586937.0,590170.0,596075.0,601347.0,602833.0
2,Fryslân (PV),Bevolking aan het einde van de periode,aantal,639787.0,642066.0,642977.0,642230.0,642209.0,643189.0,644811.0,...,646040.0,646874.0,647268.0,647672.0,649957.0,651435.0,654019.0,659551.0,661904.0,664222.0
3,Drenthe (PV),Bevolking aan het einde van de periode,aantal,481254.0,482415.0,483369.0,484481.0,486197.0,488135.0,489918.0,...,488629.0,491792.0,492100.0,492167.0,493682.0,494771.0,497743.0,502051.0,504129.0,506529.0
4,Overijssel (PV),Bevolking aan het einde van de periode,aantal,1100677.0,1105512.0,1109432.0,1113529.0,1116374.0,1119994.0,1125435.0,...,1144280.0,1147687.0,1151501.0,1156431.0,1162406.0,1166533.0,1171910.0,1184333.0,1189033.0,1195789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,Zutphen,Bevolking aan het einde van de periode,aantal,36924.0,36552.0,36936.0,46642.0,46635.0,46762.0,46953.0,...,46997.0,47340.0,47537.0,47609.0,47934.0,48111.0,48330.0,48510.0,48752.0,49028.0
566,Zwartewaterland,Bevolking aan het einde van de periode,aantal,22206.0,22034.0,21877.0,21872.0,21919.0,21930.0,21944.0,...,22278.0,22309.0,22468.0,22503.0,22685.0,22823.0,23012.0,23368.0,23448.0,23507.0
567,Zwijndrecht,Bevolking aan het einde van de periode,aantal,41374.0,45384.0,45209.0,44588.0,44400.0,44472.0,44312.0,...,44454.0,44417.0,44586.0,44639.0,44737.0,44775.0,44789.0,45018.0,44874.0,44669.0
568,Zwolle,Bevolking aan het einde van de periode,aantal,109955.0,110880.0,111900.0,113078.0,114635.0,116365.0,117703.0,...,124896.0,125548.0,126116.0,127497.0,128840.0,129840.0,130668.0,132411.0,133141.0,133810.0


## Cleaning

In [372]:
## Drop last row, invalid
popdata_df = popdata_df[:-1]

## Drop onderwerp and unnamed columns, not needed
popdata_df = popdata_df.drop(columns=['Onderwerp', 'Unnamed: 2'])      

## Rename columns
popdata_df.rename(columns={"Regio's":"region"}, inplace=True)
popdata_df

,region,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Nederland,16192572.0,16258032.0,16305526.0,16334210.0,16357992.0,16405399.0,16485787.0,16574989.0,16655799.0,...,16979120.0,17081507.0,17181084.0,17282163.0,17407585.0,17475415.0,17590672.0,17811291.0,17942942.0,18044027.0
1,Groningen (PV),572997.0,574384.0,575072.0,574042.0,573614.0,573459.0,574092.0,576668.0,579036.0,...,583721.0,583581.0,582944.0,583990.0,585866.0,586937.0,590170.0,596075.0,601347.0,602833.0
2,Fryslân (PV),639787.0,642066.0,642977.0,642230.0,642209.0,643189.0,644811.0,646305.0,647282.0,...,646040.0,646874.0,647268.0,647672.0,649957.0,651435.0,654019.0,659551.0,661904.0,664222.0
3,Drenthe (PV),481254.0,482415.0,483369.0,484481.0,486197.0,488135.0,489918.0,490981.0,491411.0,...,488629.0,491792.0,492100.0,492167.0,493682.0,494771.0,497743.0,502051.0,504129.0,506529.0
4,Overijssel (PV),1100677.0,1105512.0,1109432.0,1113529.0,1116374.0,1119994.0,1125435.0,1130345.0,1134465.0,...,1144280.0,1147687.0,1151501.0,1156431.0,1162406.0,1166533.0,1171910.0,1184333.0,1189033.0,1195789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,Zundert,20386.0,20431.0,20554.0,20797.0,20849.0,20947.0,21083.0,21025.0,21163.0,...,21488.0,21657.0,21525.0,21612.0,21829.0,21988.0,22260.0,22518.0,22553.0,22809.0
565,Zutphen,36924.0,36552.0,36936.0,46642.0,46635.0,46762.0,46953.0,46870.0,47084.0,...,46997.0,47340.0,47537.0,47609.0,47934.0,48111.0,48330.0,48510.0,48752.0,49028.0
566,Zwartewaterland,22206.0,22034.0,21877.0,21872.0,21919.0,21930.0,21944.0,21935.0,22018.0,...,22278.0,22309.0,22468.0,22503.0,22685.0,22823.0,23012.0,23368.0,23448.0,23507.0
567,Zwijndrecht,41374.0,45384.0,45209.0,44588.0,44400.0,44472.0,44312.0,44404.0,44445.0,...,44454.0,44417.0,44586.0,44639.0,44737.0,44775.0,44789.0,45018.0,44874.0,44669.0


In [373]:
## Why missing 2022 data?
popdata_df.columns

Index(['region', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')

In [374]:
print(crime_df.dtypes)
print(popdata_df.dtypes)

ID                              int64
SoortMisdrijf                  object
RegioS                         object
Perioden                       object
GeregistreerdeMisdrijven_1    float64
dtype: object
region     object
2002      float64
2003      float64
2004      float64
2005      float64
2006      float64
2007      float64
2008      float64
2009      float64
2010      float64
2011      float64
2012      float64
2013      float64
2014      float64
2015      float64
2016      float64
2017      float64
2018      float64
2019      float64
2020      float64
2021      float64
2022      float64
2023      float64
2024      float64
dtype: object


## Extract crime metadata

In [375]:
## Check unique codes per crime, region and period
print('Unique crime codes:')
print(crime_df['SoortMisdrijf'].unique())
print('Unique gemeente codes:')
print(crime_df['RegioS'].unique())
print('Unique year codes:')
print(crime_df['Perioden'].unique())

Unique crime codes:
['0.0.0 ' '1.1.1 ' '1.1.2 ' '1.2.1 ' '1.2.2 ' '1.2.3 ' '1.2.4 ' '1.2.5 '
 '1.3.1 ' '1.4.1 ' '1.4.2 ' '1.4.3 ' '1.4.4 ' '1.4.5 ' '1.4.6 ' '1.4.7 '
 '1.5.2 ' '1.6.1 ' '1.6.2 ' '1.6.3 ' '2.1.1 ' '2.2.1 ' '2.4.1 ' '2.4.2 '
 '2.5.1 ' '2.5.2 ' '2.6.1 ' '2.6.2 ' '2.6.3 ' '2.6.4 ' '2.6.5 ' '2.6.7 '
 '2.6.8 ' '2.6.9 ' '2.6.10' '2.6.11' '2.6.12' '2.6.13' '2.6.14' '2.7.2 '
 '2.7.3 ' '3.1.1 ' '3.1.2 ' '3.1.3 ' '3.2.1 ' '3.2.2 ' '3.3.2 ' '3.3.5 '
 '3.4.2 ' '3.5.2 ' '3.5.5 ' '3.6.4 ' '3.7.1 ' '3.7.2 ' '3.7.3 ' '3.7.4 '
 '3.9.1 ' '3.9.2 ' '3.9.3 ']
Unique gemeente codes:
['NL01  ' 'PV20  ' 'PV21  ' 'PV22  ' 'PV23  ' 'PV24  ' 'PV25  ' 'PV26  '
 'PV27  ' 'PV28  ' 'PV29  ' 'PV30  ' 'PV31  ' 'PV99  ' 'GM1680' 'GM0358'
 'GM0197' 'GM0059' 'GM0482' 'GM0613' 'GM0361' 'GM0141' 'GM0034' 'GM0484'
 'GM1723' 'GM1959' 'GM0060' 'GM0307' 'GM0362' 'GM0363' 'GM0200' 'GM0202'
 'GM0106' 'GM0743' 'GM0744' 'GM0308' 'GM0489' 'GM0203' 'GM0888' 'GM1954'
 'GM0889' 'GM1945' 'GM1724' 'GM0893' 'GM0373' 'GM074

In [376]:
crime_codes = list(crime_df['SoortMisdrijf'].unique())
gemeente_codes = list(crime_df['RegioS'].unique())
year_codes = list(crime_df['Perioden'].unique())
## Combined all codes in one list for iteration
code_list = crime_codes + gemeente_codes + year_codes
code_list

['0.0.0 ',
 '1.1.1 ',
 '1.1.2 ',
 '1.2.1 ',
 '1.2.2 ',
 '1.2.3 ',
 '1.2.4 ',
 '1.2.5 ',
 '1.3.1 ',
 '1.4.1 ',
 '1.4.2 ',
 '1.4.3 ',
 '1.4.4 ',
 '1.4.5 ',
 '1.4.6 ',
 '1.4.7 ',
 '1.5.2 ',
 '1.6.1 ',
 '1.6.2 ',
 '1.6.3 ',
 '2.1.1 ',
 '2.2.1 ',
 '2.4.1 ',
 '2.4.2 ',
 '2.5.1 ',
 '2.5.2 ',
 '2.6.1 ',
 '2.6.2 ',
 '2.6.3 ',
 '2.6.4 ',
 '2.6.5 ',
 '2.6.7 ',
 '2.6.8 ',
 '2.6.9 ',
 '2.6.10',
 '2.6.11',
 '2.6.12',
 '2.6.13',
 '2.6.14',
 '2.7.2 ',
 '2.7.3 ',
 '3.1.1 ',
 '3.1.2 ',
 '3.1.3 ',
 '3.2.1 ',
 '3.2.2 ',
 '3.3.2 ',
 '3.3.5 ',
 '3.4.2 ',
 '3.5.2 ',
 '3.5.5 ',
 '3.6.4 ',
 '3.7.1 ',
 '3.7.2 ',
 '3.7.3 ',
 '3.7.4 ',
 '3.9.1 ',
 '3.9.2 ',
 '3.9.3 ',
 'NL01  ',
 'PV20  ',
 'PV21  ',
 'PV22  ',
 'PV23  ',
 'PV24  ',
 'PV25  ',
 'PV26  ',
 'PV27  ',
 'PV28  ',
 'PV29  ',
 'PV30  ',
 'PV31  ',
 'PV99  ',
 'GM1680',
 'GM0358',
 'GM0197',
 'GM0059',
 'GM0482',
 'GM0613',
 'GM0361',
 'GM0141',
 'GM0034',
 'GM0484',
 'GM1723',
 'GM1959',
 'GM0060',
 'GM0307',
 'GM0362',
 'GM0363',
 'GM0200',
 'GM0202',

In [377]:
## Create dictionary with each code and its meaning
metadata_ids = metadata_sub.loc[:,'ID']
metadata_titles = metadata_sub.loc[:,'Title']

codes_dict = {}

for i, t in zip(metadata_ids, metadata_titles):
    if i in code_list:
        ## Remove the code from the crime name
        codes_dict[i] = t
        
codes_dict

{'0.0.0 ': 'Totaal misdrijven',
 '1.1.1 ': '1.1.1 Diefstal/inbraak woning',
 '1.1.2 ': '1.1.2 Diefstal/inbraak box/garage/schuur',
 '1.2.1 ': '1.2.1 Diefstal uit/vanaf motorvoertuigen',
 '1.2.2 ': '1.2.2 Diefstal van motorvoertuigen',
 '1.2.3 ': '1.2.3 Diefstal van brom-, snor-, fietsen',
 '1.2.4 ': '1.2.4 Zakkenrollerij',
 '1.2.5 ': '1.2.5 Diefstal af/uit/van ov. voertuigen',
 '1.3.1 ': '1.3.1 Ongevallen (weg)',
 '1.4.1 ': '1.4.1 Zedendelicten',
 '1.4.2 ': '1.4.2 Moord, doodslag',
 '1.4.3 ': '1.4.3 Openlijk geweld (persoon)',
 '1.4.4 ': '1.4.4 Bedreiging',
 '1.4.5 ': '1.4.5 Mishandeling',
 '1.4.6 ': '1.4.6 Straatroof',
 '1.4.7 ': '1.4.7 Overval',
 '1.5.2 ': '1.5.2 Diefstallen (water)',
 '1.6.1 ': '1.6.1 Brand/ontploffing',
 '1.6.2 ': '1.6.2 Overige vermogensdelicten',
 '1.6.3 ': '1.6.3 Mensenhandel',
 '2.1.1 ': '2.1.1 Drugs/drankoverlast',
 '2.2.1 ': '2.2.1 Vernieling cq. zaakbeschadiging',
 '2.4.1 ': '2.4.1 Burengerucht (relatieproblemen)',
 '2.4.2 ': '2.4.2 Huisvredebreuk',
 '2.5.1 

In [378]:
## Replace all codes with their meanings in the crime data
crime_df = crime_df.map(lambda x: codes_dict.get(x,x))
crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1
0,12,Totaal misdrijven,Nederland,2012,1127693.0
1,25,Totaal misdrijven,Nederland,2013,1078652.0
2,38,Totaal misdrijven,Nederland,2014,1002641.0
3,51,Totaal misdrijven,Nederland,2015,961210.0
4,64,Totaal misdrijven,Nederland,2016,914206.0
...,...,...,...,...,...
506923,3980515,3.9.3 Fraude (overig),Gemeenten; niet in te delen,2025 juli,NaN
506924,3980516,3.9.3 Fraude (overig),Gemeenten; niet in te delen,2025 augustus,NaN
506925,3980517,3.9.3 Fraude (overig),Gemeenten; niet in te delen,2025 september,NaN
506926,3980518,3.9.3 Fraude (overig),Gemeenten; niet in te delen,2025 oktober,NaN


In [379]:
## Strip code numbers from crime names
new_values = []
for i in list(crime_df['SoortMisdrijf']):
    i = re.sub(r'\d.', '', i)
    new_values.append(i)

crime_df['SoortMisdrijf'] = new_values

crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1
0,12,Totaal misdrijven,Nederland,2012,1127693.0
1,25,Totaal misdrijven,Nederland,2013,1078652.0
2,38,Totaal misdrijven,Nederland,2014,1002641.0
3,51,Totaal misdrijven,Nederland,2015,961210.0
4,64,Totaal misdrijven,Nederland,2016,914206.0
...,...,...,...,...,...
506923,3980515,Fraude (overig),Gemeenten; niet in te delen,2025 juli,NaN
506924,3980516,Fraude (overig),Gemeenten; niet in te delen,2025 augustus,NaN
506925,3980517,Fraude (overig),Gemeenten; niet in te delen,2025 september,NaN
506926,3980518,Fraude (overig),Gemeenten; niet in te delen,2025 oktober,NaN


In [380]:
## Rename columns
crime_df.rename(columns={'ID':'id','SoortMisdrijf':'misdrijf', 'RegioS':'region', 'Perioden':'period', 'GeregistreerdeMisdrijven_1':'aantal_misdrijven'},
                inplace=True)

crime_df

,id,misdrijf,region,period,aantal_misdrijven
0,12,Totaal misdrijven,Nederland,2012,1127693.0
1,25,Totaal misdrijven,Nederland,2013,1078652.0
2,38,Totaal misdrijven,Nederland,2014,1002641.0
3,51,Totaal misdrijven,Nederland,2015,961210.0
4,64,Totaal misdrijven,Nederland,2016,914206.0
...,...,...,...,...,...
506923,3980515,Fraude (overig),Gemeenten; niet in te delen,2025 juli,NaN
506924,3980516,Fraude (overig),Gemeenten; niet in te delen,2025 augustus,NaN
506925,3980517,Fraude (overig),Gemeenten; niet in te delen,2025 september,NaN
506926,3980518,Fraude (overig),Gemeenten; niet in te delen,2025 oktober,NaN


In [381]:
## Check uniqe values per column
for i in crime_df.columns:
    print(crime_df[i].unique())

[     12      25      38 ... 3980517 3980518 3980519]
['Totaal misdrijven' 'Diefstal/inbraak woning'
 'Diefstal/inbraak box/garage/schuur' 'Diefstal uit/vanaf motorvoertuigen'
 'Diefstal van motorvoertuigen' 'Diefstal van brom-, snor-, fietsen'
 'Zakkenrollerij' 'Diefstal af/uit/van ov. voertuigen' 'Ongevallen (weg)'
 'Zedendelicten' 'Moord, doodslag' 'Openlijk geweld (persoon)'
 'Bedreiging' 'Mishandeling' 'Straatroof' 'Overval' 'Diefstallen (water)'
 'Brand/ontploffing' 'Overige vermogensdelicten' 'Mensenhandel'
 'Drugs/drankoverlast' 'Vernieling cq. zaakbeschadiging'
 'Burengerucht (relatieproblemen)' 'Huisvredebreuk'
 'Diefstal/inbraak bedrijven enz.' 'Winkeldiefstal'
 'Inrichting Wet Milieubeheer' 'Bodem' 'Water' 'Afval' 'Bouwstoffen'
 'Mest' 'Transport gevaarlijke stoffen' 'Vuurwerk' ' Bestrijdingsmiddelen'
 ' Natuur en landschap' ' Ruimtelijke ordening' ' Dieren'
 ' Voedselveiligheid' 'Bijzondere wetten' 'Leefbaarheid (overig)'
 'Drugshandel' 'Mensensmokkel' 'Wapenhandel' 'Kinde

In [382]:
## Compare to unique regions in popdata_df
print(popdata_df['region'].unique())

['Nederland' 'Groningen (PV)' 'Fryslân (PV)' 'Drenthe (PV)'
 'Overijssel (PV)' 'Flevoland (PV)' 'Gelderland (PV)' 'Utrecht (PV)'
 'Noord-Holland (PV)' 'Zuid-Holland (PV)' 'Zeeland (PV)'
 'Noord-Brabant (PV)' 'Limburg (PV)' 'Aa en Hunze' 'Aalburg' 'Aalsmeer'
 'Aalten' 'Ter Aar' 'Abcoude' 'Achtkarspelen' 'Alblasserdam'
 'Albrandswaard' 'Alkemade' 'Alkmaar' 'Almelo' 'Almere'
 'Alphen aan den Rijn' 'Alphen-Chaam' 'Altena' 'Ambt Montfort' 'Ameland'
 'Amerongen' 'Amersfoort' 'Amstelveen' 'Amsterdam' 'Andijk' 'Angerlo'
 'Anna Paulowna' 'Apeldoorn' 'Appingedam' 'Arcen en Velden' 'Arnhem'
 'Assen' 'Asten' 'Axel' 'Baarle-Nassau' 'Baarn' 'Barendrecht' 'Barneveld'
 'Bathmen' 'Bedum' 'Beek (L.)' 'Beekdaelen' 'Beemster' 'Beesel'
 'Bellingwedde' 'Bemmel' 'Bennebroek' 'Berg en Dal' 'Bergambacht'
 'Bergeijk' 'Bergen (L.)' 'Bergen (NH.)' 'Bergen op Zoom' 'Bergh'
 'Bergschenhoek' 'Berkel en Rodenrijs' 'Berkelland' 'Bernheze' 'Bernisse'
 'Best' 'Beuningen' 'Beverwijk' 'het Bildt' 'De Bilt' 'Binnenmaas'
 '

## Calculate per capita crime

In [383]:
## Check df
popdata_df.head()

,region,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Nederland,16192572.0,16258032.0,16305526.0,16334210.0,16357992.0,16405399.0,16485787.0,16574989.0,16655799.0,...,16979120.0,17081507.0,17181084.0,17282163.0,17407585.0,17475415.0,17590672.0,17811291.0,17942942.0,18044027.0
1,Groningen (PV),572997.0,574384.0,575072.0,574042.0,573614.0,573459.0,574092.0,576668.0,579036.0,...,583721.0,583581.0,582944.0,583990.0,585866.0,586937.0,590170.0,596075.0,601347.0,602833.0
2,Fryslân (PV),639787.0,642066.0,642977.0,642230.0,642209.0,643189.0,644811.0,646305.0,647282.0,...,646040.0,646874.0,647268.0,647672.0,649957.0,651435.0,654019.0,659551.0,661904.0,664222.0
3,Drenthe (PV),481254.0,482415.0,483369.0,484481.0,486197.0,488135.0,489918.0,490981.0,491411.0,...,488629.0,491792.0,492100.0,492167.0,493682.0,494771.0,497743.0,502051.0,504129.0,506529.0
4,Overijssel (PV),1100677.0,1105512.0,1109432.0,1113529.0,1116374.0,1119994.0,1125435.0,1130345.0,1134465.0,...,1144280.0,1147687.0,1151501.0,1156431.0,1162406.0,1166533.0,1171910.0,1184333.0,1189033.0,1195789.0


In [384]:
## Check df
crime_df.head()

,id,misdrijf,region,period,aantal_misdrijven
0,12,Totaal misdrijven,Nederland,2012,1127693.0
1,25,Totaal misdrijven,Nederland,2013,1078652.0
2,38,Totaal misdrijven,Nederland,2014,1002641.0
3,51,Totaal misdrijven,Nederland,2015,961210.0
4,64,Totaal misdrijven,Nederland,2016,914206.0


In [385]:
## Divide population values by 1000 for easier per capita calculations
divide_by = popdata_df.iloc[:,1:].apply(lambda x: x/1000)
divide_by.head()

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,16192.572,16258.032,16305.526,16334.210,16357.992,16405.399,16485.787,16574.989,16655.799,16730.348,...,16979.120,17081.507,17181.084,17282.163,17407.585,17475.415,17590.672,17811.291,17942.942,18044.027
1,572.997,574.384,575.072,574.042,573.614,573.459,574.092,576.668,579.036,580.875,...,583.721,583.581,582.944,583.990,585.866,586.937,590.170,596.075,601.347,602.833
2,639.787,642.066,642.977,642.230,642.209,643.189,644.811,646.305,647.282,647.214,...,646.040,646.874,647.268,647.672,649.957,651.435,654.019,659.551,661.904,664.222
3,481.254,482.415,483.369,484.481,486.197,488.135,489.918,490.981,491.411,490.807,...,488.629,491.792,492.100,492.167,493.682,494.771,497.743,502.051,504.129,506.529
4,1100.677,1105.512,1109.432,1113.529,1116.374,1119.994,1125.435,1130.345,1134.465,1137.668,...,1144.280,1147.687,1151.501,1156.431,1162.406,1166.533,1171.910,1184.333,1189.033,1195.789


In [386]:
## Concat per 1000 values back to region column
pop_per1000 = pd.concat([popdata_df.iloc[:,0], divide_by], axis=1)

pop_per1000.head()


,region,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Nederland,16192.572,16258.032,16305.526,16334.210,16357.992,16405.399,16485.787,16574.989,16655.799,...,16979.120,17081.507,17181.084,17282.163,17407.585,17475.415,17590.672,17811.291,17942.942,18044.027
1,Groningen (PV),572.997,574.384,575.072,574.042,573.614,573.459,574.092,576.668,579.036,...,583.721,583.581,582.944,583.990,585.866,586.937,590.170,596.075,601.347,602.833
2,Fryslân (PV),639.787,642.066,642.977,642.230,642.209,643.189,644.811,646.305,647.282,...,646.040,646.874,647.268,647.672,649.957,651.435,654.019,659.551,661.904,664.222
3,Drenthe (PV),481.254,482.415,483.369,484.481,486.197,488.135,489.918,490.981,491.411,...,488.629,491.792,492.100,492.167,493.682,494.771,497.743,502.051,504.129,506.529
4,Overijssel (PV),1100.677,1105.512,1109.432,1113.529,1116.374,1119.994,1125.435,1130.345,1134.465,...,1144.280,1147.687,1151.501,1156.431,1162.406,1166.533,1171.910,1184.333,1189.033,1195.789


In [387]:
## Melt dataframe to match crime_df structure
per1000_long = pd.melt(
    pop_per1000,
    id_vars=['region'],
    var_name='period',
    value_name='population'
)

per1000_long.head()

,region,period,population
0,Nederland,2002,16192.572
1,Groningen (PV),2002,572.997
2,Fryslân (PV),2002,639.787
3,Drenthe (PV),2002,481.254
4,Overijssel (PV),2002,1100.677


In [388]:
## Merge two dataframes by region and period
merged_df = crime_df.merge(per1000_long, on=['region','period'])
## Calculate misdrijven_per_cap column 
merged_df['misdrijven_per_cap'] = round(merged_df['aantal_misdrijven']/merged_df['population'],2)
merged_df.rename(columns={'population':'pop_per_1000'}, inplace=True)
# merged_df.head(50)
print(merged_df.shape)

(272285, 7)


In [389]:
merged_df.head()

,id,misdrijf,region,period,aantal_misdrijven,pop_per_1000,misdrijven_per_cap
0,12,Totaal misdrijven,Nederland,2012,1127693.0,16779.575,67.21
1,25,Totaal misdrijven,Nederland,2013,1078652.0,16829.289,64.09
2,38,Totaal misdrijven,Nederland,2014,1002641.0,16900.726,59.33
3,51,Totaal misdrijven,Nederland,2015,961210.0,16979.120,56.61
4,64,Totaal misdrijven,Nederland,2016,914206.0,17081.507,53.52


## Create crime categories

In [390]:
## Check unique crime types
print(crime_df['misdrijf'].unique())

['Totaal misdrijven' 'Diefstal/inbraak woning'
 'Diefstal/inbraak box/garage/schuur' 'Diefstal uit/vanaf motorvoertuigen'
 'Diefstal van motorvoertuigen' 'Diefstal van brom-, snor-, fietsen'
 'Zakkenrollerij' 'Diefstal af/uit/van ov. voertuigen' 'Ongevallen (weg)'
 'Zedendelicten' 'Moord, doodslag' 'Openlijk geweld (persoon)'
 'Bedreiging' 'Mishandeling' 'Straatroof' 'Overval' 'Diefstallen (water)'
 'Brand/ontploffing' 'Overige vermogensdelicten' 'Mensenhandel'
 'Drugs/drankoverlast' 'Vernieling cq. zaakbeschadiging'
 'Burengerucht (relatieproblemen)' 'Huisvredebreuk'
 'Diefstal/inbraak bedrijven enz.' 'Winkeldiefstal'
 'Inrichting Wet Milieubeheer' 'Bodem' 'Water' 'Afval' 'Bouwstoffen'
 'Mest' 'Transport gevaarlijke stoffen' 'Vuurwerk' ' Bestrijdingsmiddelen'
 ' Natuur en landschap' ' Ruimtelijke ordening' ' Dieren'
 ' Voedselveiligheid' 'Bijzondere wetten' 'Leefbaarheid (overig)'
 'Drugshandel' 'Mensensmokkel' 'Wapenhandel' 'Kinderporno'
 'Kinderprostitutie' 'Onder invloed (lucht)' '

In [391]:
violent_crime = ['Zedendelicten','Moord, doodslag', 'Openlijk geweld (persoon)',
 'Bedreiging',
 'Mishandeling',
 'Straatroof',
 'Overval','Mensenhandel','Mensensmokkel', 'Kinderporno',
 'Kinderprostitutie', 'Wapenhandel']

explosive_crime = ['Brand/ontploffing', 'Vuurwerk']

env_crime = ['Inrichting Wet Milieubeheer', 'Bodem', 'Water', 'Afval', 'Bouwstoffen',
 'Mest', 'Transport gevaarlijke stoffen', 'Lucht (overig)',' Natuur en landschap', ' Dieren']

thievery = ['Diefstal/inbraak box/garage/schuur', 'Diefstal uit/vanaf motorvoertuigen', 'Diefstal van motorvoertuigen', 
'Diefstal van brom-, snor-, fietsen', 'Zakkenrollerij', 'Diefstal af/uit/van ov. voertuigen', 
'Diefstal/inbraak bedrijven enz.', 'Winkeldiefstal']

traffic = ['Ongevallen (weg)', 'Onder invloed (weg)', 'Weg (overig)' ]

## Create city & regional groupings

In [392]:
## Check gemeente names
crime_df['region'].unique()

array(['Nederland', 'Groningen (PV)', 'Fryslân (PV)', 'Drenthe (PV)',
       'Overijssel (PV)', 'Flevoland (PV)', 'Gelderland (PV)',
       'Utrecht (PV)', 'Noord-Holland (PV)', 'Zuid-Holland (PV)',
       'Zeeland (PV)', 'Noord-Brabant (PV)', 'Limburg (PV)',
       'Niet in te delen (PV)', 'Aa en Hunze', 'Aalsmeer', 'Aalten',
       'Achtkarspelen', 'Alblasserdam', 'Albrandswaard', 'Alkmaar',
       'Almelo', 'Almere', 'Alphen aan den Rijn', 'Alphen-Chaam',
       'Altena', 'Ameland', 'Amersfoort', 'Amstelveen', 'Amsterdam',
       'Apeldoorn', 'Arnhem', 'Assen', 'Asten', 'Baarle-Nassau', 'Baarn',
       'Barendrecht', 'Barneveld', 'Beek (L.)', 'Beekdaelen', 'Beesel',
       'Berg en Dal', 'Bergeijk', 'Bergen (L.)', 'Bergen (NH.)',
       'Bergen op Zoom', 'Berkelland', 'Bernheze', 'Best', 'Beuningen',
       'Beverwijk', 'De Bilt', 'Bladel', 'Blaricum', 'Bloemendaal',
       'Bodegraven-Reeuwijk', 'Boekel', 'Borger-Odoorn', 'Borne',
       'Borsele', 'Boxtel', 'Breda', 'Bronckhorst',

In [393]:
## Create city groupings of interest for later analysis
brabant_steden = ['Eindhoven', 'Tilburg', 'Breda', "'s-Hertogenbosch"]
randstad_steden = ['Amsterdam', 'Utrecht (gemeente)', 'Rotterdam', "'s-Gravenhage (gemmente)", 'Haarlem']
grootste_steden = ['Amsterdam', 'Utrecht (gemeente)', 'Rotterdam', "'s-Gravenhage (gemmente)", 
                   'Eindhoven', 'Tilburg', 'Breda', "'s-Hertogenbosch", 'Arnhem', 'Nijmegen', 'Groningen (gemeente)']

In [394]:
## Create separate dfs for regions, gemeenten and whole country for later analysis
regions_df = merged_df[merged_df['region'].str.contains('(PV)')]
country_df = merged_df[merged_df['region'] == 'Nederland']
gemeenten_df = merged_df[(~merged_df['region'].isin(country_df['region'])) & (~merged_df['region'].isin(regions_df['region']))]

C:\Users\chels\AppData\Local\Temp\ipykernel_40800\1773671532.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_df = merged_df[merged_df['region'].str.contains('(PV)')]


In [395]:
gemeenten_df

,id,misdrijf,region,period,aantal_misdrijven,pop_per_1000,misdrijven_per_cap
169,3612,Totaal misdrijven,Aa en Hunze,2012,925.0,25.541,36.22
170,3625,Totaal misdrijven,Aa en Hunze,2013,755.0,25.357,29.77
171,3638,Totaal misdrijven,Aa en Hunze,2014,664.0,25.203,26.35
172,3651,Totaal misdrijven,Aa en Hunze,2015,634.0,25.243,25.12
173,3664,Totaal misdrijven,Aa en Hunze,2016,616.0,25.286,24.36
...,...,...,...,...,...,...,...
272280,3980096,Fraude (overig),Zwolle,2020,28.0,129.840,0.22
272281,3980109,Fraude (overig),Zwolle,2021,8.0,130.668,0.06
272282,3980122,Fraude (overig),Zwolle,2022,8.0,132.411,0.06
272283,3980135,Fraude (overig),Zwolle,2023,7.0,133.141,0.05


# Total crime over time

## Whole country

In [396]:
# Filter for total crime in all of the Netherlands over time
total_crimes_df = country_df[(country_df['misdrijf'] == 'Totaal misdrijven')]

total_crimes_df

,id,misdrijf,region,period,aantal_misdrijven,pop_per_1000,misdrijven_per_cap
0,12,Totaal misdrijven,Nederland,2012,1127693.0,16779.575,67.21
1,25,Totaal misdrijven,Nederland,2013,1078652.0,16829.289,64.09
2,38,Totaal misdrijven,Nederland,2014,1002641.0,16900.726,59.33
3,51,Totaal misdrijven,Nederland,2015,961210.0,16979.120,56.61
4,64,Totaal misdrijven,Nederland,2016,914206.0,17081.507,53.52
5,77,Totaal misdrijven,Nederland,2017,816608.0,17181.084,47.53
6,90,Totaal misdrijven,Nederland,2018,770418.0,17282.163,44.58
7,103,Totaal misdrijven,Nederland,2019,803632.0,17407.585,46.17
8,116,Totaal misdrijven,Nederland,2020,797802.0,17475.415,45.65
9,129,Totaal misdrijven,Nederland,2021,743015.0,17590.672,42.24


In [397]:
# Plot
sns.lineplot(
    data=total_crimes_df,
    x='period',
    y='misdrijven_per_cap',
    marker='o'
)
plt.title('Total Crimes per Capita by Year (Netherlands)')
plt.xticks(rotation=45)
plt.show()


## Top cities for crime over time

In [420]:
# Find the top 20 cities with the highest overall crime per 1000 people on average over time
gemeenten_totals = gemeenten_df[gemeenten_df['misdrijf'] == 'Totaal misdrijven']

top20_gemeenten = gemeenten_totals[['region','misdrijven_per_cap']].groupby(by=['region']).mean().sort_values(
    by=['misdrijven_per_cap'], axis=0, ascending=False).head(20)

top20_gemeenten

,misdrijven_per_cap
region,
Amsterdam,99.143077
Rotterdam,87.591538
Eindhoven,85.519231
Utrecht (gemeente),79.039231
Maastricht,77.980000
's-Gravenhage (gemeente),76.937692
Arnhem,75.890769
Roermond,75.713846
Ouder-Amstel,73.721538


In [436]:
# Create a line chart showing the top 10 cities total crime progression
plot_top10 = gemeenten_df[(gemeenten_df['region'].isin(top20_gemeenten.index[:10])) & (gemeenten_df['misdrijf']=='Totaal misdrijven')]

plot_top10


sns.lineplot(
    data=plot_top10,
    x='period',
    y='misdrijven_per_cap',
    hue='region',
    marker='o'
)

plt.title('Total crime per 1000 people over time \n for the top 10 gemeenten by average total crime')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 3) Find the 10 cities with the highest crime in 2024


In [ ]:
# 4) Make a bar chart

In [ ]:
# Histogram of distribution of total crime figures per gemeente in 2024
# Break out box with max, min and median values (plus cities associated with max and min)

## Regional comparison

In [418]:
plot_regions = regions_df[regions_df['misdrijf']=='Totaal misdrijven']
plot_regions

,id,misdrijf,region,period,aantal_misdrijven,pop_per_1000,misdrijven_per_cap
13,1272,Totaal misdrijven,Groningen (PV),2012,35220.0,581.705,60.55
14,1285,Totaal misdrijven,Groningen (PV),2013,33755.0,582.728,57.93
15,1298,Totaal misdrijven,Groningen (PV),2014,30162.0,583.942,51.65
16,1311,Totaal misdrijven,Groningen (PV),2015,29796.0,583.721,51.04
17,1324,Totaal misdrijven,Groningen (PV),2016,29291.0,583.581,50.19
...,...,...,...,...,...,...,...
164,3356,Totaal misdrijven,Limburg (PV),2020,50371.0,1115.872,45.14
165,3369,Totaal misdrijven,Limburg (PV),2021,47907.0,1118.302,42.84
166,3382,Totaal misdrijven,Limburg (PV),2022,52765.0,1128.367,46.76
167,3395,Totaal misdrijven,Limburg (PV),2023,52426.0,1133.255,46.26


In [419]:
# Line chart of each region (PV) figures over time
sns.lineplot(
    data=plot_regions,
    x='period',
    y='misdrijven_per_cap',
    hue='region',
    marker='o'
)

plt.title('Total Crimes per 1000 people for each region per year')
plt.xticks(rotation=45)
plt.show()


## City group comparison

In [ ]:
# Line chart of the average for each city group (Brabant vs. Randstad) over time

# Violent crime over time

## Total over time

In [ ]:
# Line chart, single line

## Top cities over time

In [344]:
# Take top 10 cities from before and show violent crime line chart

# Comparison of crime categories

In [ ]:
## Line chart showing progression of total crime in each crime category

In [ ]:
## Bar charts showing top 5 cities per crime category in 2024

In [366]:
 ## Bar chart showing top crime type (not category) per major city (grootste steden)